<a href="https://colab.research.google.com/github/Magaldany/sankey-lulc/blob/main/sankeylulc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Dash
!pip install plotly
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [12]:
import dash
from dash import dcc, html
import plotly.graph_objects as go
import pandas as pd

# Função para processar dados de um arquivo TSV e preparar o gráfico
def process_data(file_path):
    data = pd.read_csv(file_path, delimiter='\t')  # Lê o arquivo TSV

    # Verifica se a coluna 'Weight' existe antes de prosseguir
    if 'Weight' in data.columns:
        data['Weight'] = data['Weight'].str.replace(',', '.').astype(float).astype(int)
        data = data[data['Weight'] != 0]  # Filtra linhas onde o peso é zero
    else:
        print(f"Warning: 'Weight' column not found in {file_path}")
        return [], [], []

    return data['from'].tolist(), data['to'].tolist(), data['Weight'].tolist()

def create_sankey_chart(file_path, class_color_map, title="Sankey Diagram"):
    from_nodes, to_nodes, weights = process_data(file_path)
    unique_nodes = list(set(from_nodes + to_nodes))

    # Atribui cores aos nós
    node_colors = [class_color_map.get(node, "#cfd1d1") for node in unique_nodes]  # Cor padrão cinza

    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=unique_nodes,
            color=node_colors
        ),
        link=dict(
            source=[unique_nodes.index(src) for src in from_nodes],
            target=[unique_nodes.index(tgt) for tgt in to_nodes],
            value=weights,
            color=["#cfd1d1"] * len(weights)  # Todas as ligações em cinza
        )
    ))
    fig.update_layout(title_text=title, font_size=12, margin=dict(l=100, r=100, t=40, b=100))  # Ajustando margens
    return fig

# Inicializa o aplicativo Dash
app = dash.Dash(__name__)

# Dicionário de cores por classe
class_color_map = {
    "Floresta Natural|1985": "#118504",  # Verde
    "Pastagem|2000": "#ffff69",  # Amarelo
    "Pastagem|2015": "#ffff69",  # Amarelo
    "Agricultura|1985": "#ffc0cb",  # Rosa
    "Agricultura|2000": "#ffc0cb",  # Rosa
    "Agricultura|2015": "#ffc0cb",  # Rosa
    "Área Urbana|1985": "#FF0000",  # Vermelho
    "Área Urbana|2000": "#FF0000",  # Vermelho
    "Área Urbana|2015": "#FF0000",  # Vermelho


    # Adicione mais classes e cores conforme necessário
}

# Configura o layout do aplicativo para exibir apenas o gráfico do Diagrama
app.layout = html.Div(id='box', style={
    'position': 'absolute',
    'marginTop': '-10px',
    'marginBottom': '50px',
    'font-family': 'Arial, Helvetica, sans-serif',
    'textAlign': 'center',
    'width': '100%',
    'height': 'auto',
    'overflow': 'auto'
}, children=[
    html.H1('Mudança de Uso e Cobertura da Terra', style={'color': 'black'}), # Substituir o título conforme necessário
    dcc.Graph(
        id='teorema1',
        figure=create_sankey_chart(
            "/content/teste1.tsv", # Substituir o caminho da pasta de acordo com o arquivo a ser introduzido
            class_color_map,
            title="Substituição da Floresta Natural" # Substituir o título conforme necessário
        ),
        style={'position': 'relative', 'width': '100%', 'height': '600px'}
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>